# aggregate_results_of_different_types.py - ELI5 Walkthrough
This notebook recreates `python/samples/getting_started/workflows/parallelism/aggregate_results_of_different_types.py` so you can follow the fan-out/fan-in pattern step by step.


## Big Picture
A dispatcher fans out a list of numbers to two workers (Average and Sum). Their outputs are fanned back in by an Aggregator that collects multiple result types in a single list.


## Key Ingredients
- `WorkflowBuilder.add_fan_out_edges` and `.add_fan_in_edges` express parallel branches and joins.
- Executors send messages with typed `WorkflowContext`.
- Streaming run emits `WorkflowOutputEvent` when the fan-in node yields results.


### Workflow Diagram
```mermaid
flowchart LR
    Start(["Number List"]) --> Dispatcher[[Dispatcher]]
    Dispatcher --> Average[[Average]]
    Dispatcher --> Sum[[Sum]]
    Average --> Aggregator[[Aggregator]]
    Sum --> Aggregator
    Aggregator --> Output(["[sum, average]"])
```


### Step 1: Imports and scenario overview
We load the Agent Framework primitives, configure environment variables, and keep the docstring describing the fan-out/fan-in demo.


In [16]:
# Copyright (c) Microsoft. All rights reserved.
from dotenv import load_dotenv
load_dotenv()

import asyncio
import random

from agent_framework import Executor, WorkflowBuilder, WorkflowContext, WorkflowOutputEvent, handler
from typing_extensions import Never

"""
Sample: Concurrent fan out and fan in with two different tasks that output results of different types.

Purpose:
Show how to construct a parallel branch pattern in workflows. Demonstrate:
- Fan out by targeting multiple executors from one dispatcher.
- Fan in by collecting a list of results from the executors.
- Simple tracing using AgentRunEvent to observe execution order and progress.

Prerequisites:
- Familiarity with WorkflowBuilder, executors, edges, events, and streaming runs.
"""




'\nSample: Concurrent fan out and fan in with two different tasks that output results of different types.\n\nPurpose:\nShow how to construct a parallel branch pattern in workflows. Demonstrate:\n- Fan out by targeting multiple executors from one dispatcher.\n- Fan in by collecting a list of results from the executors.\n- Simple tracing using AgentRunEvent to observe execution order and progress.\n\nPrerequisites:\n- Familiarity with WorkflowBuilder, executors, edges, events, and streaming runs.\n'

### Step 2: Dispatcher fans out the input
`Dispatcher` validates the incoming list and forwards it to each parallel worker.


In [23]:
class Dispatcher(Executor):
    """
    The sole purpose of this decorator is to dispatch the input of the workflow to
    other executors.
    """

    @handler
    async def handle(self, numbers: list[int], ctx: WorkflowContext[list[int]]):
        print(f"📤 Dispatcher [{self.id}]: Received {len(numbers)} numbers: {numbers}")
        print(f"📤 Dispatcher [{self.id}]: Input range: min={min(numbers)}, max={max(numbers)}")
        
        if not numbers:
            raise RuntimeError("Input must be a valid list of integers.")

        print(f"📤 Dispatcher [{self.id}]: Sending numbers to parallel workers...")
        await ctx.send_message(numbers)

### Step 3: Worker executors compute their metrics
`Average` and `Sum` each take the same list and emit their numeric result.


In [24]:
class Average(Executor):
    """Calculate the average of a list of integers."""

    @handler
    async def handle(self, numbers: list[int], ctx: WorkflowContext[float]):
        print(f"🧮 Average [{self.id}]: Processing {len(numbers)} numbers...")
        print(f"🧮 Average [{self.id}]: Input sum: {sum(numbers)}, count: {len(numbers)}")
        average: float = sum(numbers) / len(numbers)
        print(f"🧮 Average [{self.id}]: Calculated average = {average:.2f} (formula: {sum(numbers)}/{len(numbers)})")
        await ctx.send_message(average)

In [25]:
class Sum(Executor):
    """Calculate the sum of a list of integers."""

    @handler
    async def handle(self, numbers: list[int], ctx: WorkflowContext[int]):
        print(f"➕ Sum [{self.id}]: Processing {len(numbers)} numbers...")
        print(f"➕ Sum [{self.id}]: Input values: {numbers}")
        total: int = sum(numbers)
        print(f"➕ Sum [{self.id}]: Calculated sum = {total} (type: {type(total).__name__})")
        await ctx.send_message(total)

### Step 4: Aggregator fans the results back in
`Aggregator` receives a heterogeneous list (`int | float`) and yields it as the final workflow output.


In [26]:
class Aggregator(Executor):
    """Aggregate the results from the different tasks and yield the final output."""

    @handler
    async def handle(self, results: list[int | float], ctx: WorkflowContext[Never, list[int | float]]):
        """Receive the results from the source executors.

        The framework will automatically collect messages from the source executors
        and deliver them as a list.

        Args:
            results (list[int | float]): execution results from upstream executors.
                The type annotation must be a list of union types that the upstream
                executors will produce.
            ctx (WorkflowContext[Never, list[int | float]]): A workflow context that can yield the final output.
        """
        print(f"📊 Aggregator [{self.id}]: Received {len(results)} results from upstream workers")
        print(f"📊 Aggregator [{self.id}]: Result types: {[type(r).__name__ for r in results]}")
        print(f"📊 Aggregator [{self.id}]: Result values: {results}")
        print(f"📊 Aggregator [{self.id}]: Yielding final output (type: {type(results).__name__})...")
        await ctx.yield_output(results)

### Step 5: Wire and run the workflow
`main()` constructs the graph, executes it with random numbers, and prints the collected outputs.


In [29]:
async def main() -> None:
    # 1) Create the executors
    print("🏗️  Creating executors...")
    dispatcher = Dispatcher(id="dispatcher")
    average = Average(id="average")
    summation = Sum(id="summation")
    aggregator = Aggregator(id="aggregator")
    
    executor_names = [dispatcher.id, average.id, summation.id, aggregator.id]
    print(f"✅ Executors created: {', '.join(executor_names)} (total: {len(executor_names)})")

    # 2) Build a simple fan out and fan in workflow
    print(f"\n🔧 Building workflow...")
    workflow = (
        WorkflowBuilder()
        .set_start_executor(dispatcher)
        .add_fan_out_edges(dispatcher, [average, summation])
        .add_fan_in_edges([average, summation], aggregator)
        .build()
    )
    print(f"✅ Workflow built with fan-out/fan-in pattern")
    print(f"   - Start: {dispatcher.id}")
    print(f"   - Fan-out targets: {[average.id, summation.id]}")
    print(f"   - Fan-in source: [{average.id}, {summation.id}] -> {aggregator.id}")

    # 3) Run the workflow
    print(f"\n🎲 Generating random input data...")
    input_numbers = [random.randint(1, 100) for _ in range(16)]
    print(f"📊 Input numbers ({len(input_numbers)} values): {input_numbers}")
    print(f"📊 Input statistics: min={min(input_numbers)}, max={max(input_numbers)}, sum={sum(input_numbers)}")
    
    print(f"\n🚀 Running workflow...")
    output: list[int | float] | None = None
    event_count = 0
    
    async for event in workflow.run_stream(input_numbers):
        event_count += 1
        event_type = type(event).__name__
        print(f"⚡ Event {event_count}: {event_type}")
        
        if isinstance(event, WorkflowOutputEvent):
            output = event.data
            print(f"🎯 Workflow completed! Output received: {output} (type: {type(output).__name__})")

    if output is not None:
        print(f"\n🏆 Final Result: {output}")
        print(f"   - Sum: {output[0]} (type: {type(output[0]).__name__})")
        print(f"   - Average: {output[1]} (type: {type(output[1]).__name__})")
        print(f"   - Total events processed: {event_count}")
    else:
        print("❌ No output received from workflow")

### Step 6: Try it yourself
Use the helper below. In notebooks it awaits `main()` on the active loop; in scripts it falls back to `asyncio.run(main())`.


In [30]:
import asyncio

# Helper for notebooks vs. scripts
loop = asyncio.get_event_loop()
if loop.is_running():
    # Jupyter/VS Code notebooks already have an event loop, so await directly.
    await main()
else:
    asyncio.run(main())


🏗️  Creating executors...
✅ Executors created: dispatcher, average, summation, aggregator (total: 4)

🔧 Building workflow...
✅ Workflow built with fan-out/fan-in pattern
   - Start: dispatcher
   - Fan-out targets: ['average', 'summation']
   - Fan-in source: [average, summation] -> aggregator

🎲 Generating random input data...
📊 Input numbers (16 values): [7, 36, 46, 49, 33, 58, 1, 7, 11, 18, 81, 51, 23, 2, 9, 52]
📊 Input statistics: min=1, max=81, sum=484

🚀 Running workflow...
⚡ Event 1: WorkflowStartedEvent
⚡ Event 2: WorkflowStatusEvent
📤 Dispatcher [dispatcher]: Received 16 numbers: [7, 36, 46, 49, 33, 58, 1, 7, 11, 18, 81, 51, 23, 2, 9, 52]
📤 Dispatcher [dispatcher]: Input range: min=1, max=81
📤 Dispatcher [dispatcher]: Sending numbers to parallel workers...
⚡ Event 3: ExecutorInvokedEvent
⚡ Event 4: ExecutorCompletedEvent
🧮 Average [average]: Processing 16 numbers...
🧮 Average [average]: Input sum: 484, count: 16
🧮 Average [average]: Calculated average = 30.25 (formula: 484/16)